In [2]:
%load_ext rpy2.ipython

In [6]:
%%R
library('gridGraphics')
library('cowplot')
source('../analysis_tools/kiviat_plotter.R')


Error in library("cowplot") : there is no package called 'cowplot'


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in library("cowplot") : there is no package called 'cowplot'

  warnings.warn(x, RRuntimeWarning)
